In [5]:
pip install nltk


     |████████████████████████████████| 1.5 MB 2.6 MB/s            
     |████████████████████████████████| 749 kB 60.5 MB/s            
Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import string 
import nltk
import re
from nltk.corpus import stopwords

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import keras
from keras.preprocessing import text,sequence
from keras.models import Sequential
from keras.layers import Dense,Embedding,LSTM,Dropout

import warnings
warnings.filterwarnings('ignore')

In [7]:
from google.cloud import storage

from io import BytesIO

client = storage.Client()

bucket = "hackteam-brainpower"
file_name = "cleaned_data/part-00000-99d78a6d-8a14-4d63-abce-2724aef10f68-c000.csv"

In [8]:
bucket = client.get_bucket(bucket)
blob = bucket.get_blob(file_name)
content = blob.download_as_string()

df = pd.read_csv(BytesIO(content),on_bad_lines='skip',names=["url", "type", "source", "domain","content"])

In [12]:

df = df[["content", 'type']]
df

#Alter the rows 
df_1 = df[df['type'] == 'conspiracy']
print(df_1.shape)
df_2 = df[df['type'] == 'fake']
print(df_2.shape)
df_3 = df[df['type'] == 'unreliable']
print(df_3.shape)
df_4 = df[df['type'] == 'clickbait']
print(df_4.shape)
df_5 = df[df['type'] == 'reliable']
print(df_5.shape)
df_new = pd.concat([df_1, df_2,df_3, df_4, df_5], axis=0)
df_new = df_new.reset_index()
df_new= df_new.copy()
df = df_new
df.drop(["index"], axis=1)
df.head()

# df = df.sample(frac =0.025)


df = df[["content", 'type']]

df.head(3)

df = df.dropna()
df.shape

#def clean_text(text):
 #   '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
 #   and remove words containing numbers.'''
 #   text = str(text).lower()
 #   text = re.sub('\[.*?\]', '', text)
#    text = re.sub('https?://\S+|www\.\S+', '', text)
 #   text = re.sub('<.*?>+', '', text)
  #  text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
  #  text = re.sub('\n', '', text)
  #  text = re.sub('\w*\d\w*', '', text)
  #  return text



# def clean_text_(text):
#     '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
#     and remove words containing numbers.'''
#     text = str(text).lower()
#     text = re.sub('\[.*?\]', '', text)
#     text = re.sub('https?://\S+|www\.\S+', '', text)
#     text = re.sub('<.*?>+', '', text)
#     text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
#     text = re.sub('\n', '', text)
#     text = re.sub('\w*\d\w*', '', text)
#     return text
# tt = "Hello wOrld !! WelCome to ( PyTHon's tuTorial ) we wiLL stArt at SHARP 9:00 o'clock"
# new = clean_text(tt)
# import nltk
# from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords
# stop_words = set(stopwords.words('english'))
# word_tokens = word_tokenize(new)
# filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
# print(filtered_sentence)
# ps = PorterStemmer()
# for i in filtered_sentence:
#     print(ps.stem(i))

df.type.value_counts()

df.type = df.type.replace("conspiracy", "fake").replace("unreliable", "fake").replace("clickbait", "fake")
df.type = df.type.replace('fake', 0).replace('reliable',1)
df.type.value_counts()

#df['content'] = df['content'].apply(lambda x:clean_text(x))



max_size = 200
df['content'] = df['content'].apply(lambda x: ' '.join(x.split(maxsplit=max_size)[:max_size]))
df['content']

## GETTING INDEPENDENT FEATURES
x= df.drop('type',axis=1)

# DEPENDENT FEATURE OR TARGET FEATURE
y=df['type']

# LOOKING AT THE SHAPE OF OUR IINDEPENDENT FEATURES DATASET
print(x.shape)

# LOOKING AT THE SHAPE OF OUR DEPENDENT FEATURE
print(y.shape)



from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

# vocabularry size
voc_size=40000

# COPYING OUR INDEPENDENT FEATURE DATASET 'x' INTO NEW VARIABLE 'messages'
messages= x.copy()
messages.reset_index(inplace=True)



# DOWNLOADING THE STOPWORDS
nltk.download('stopwords')


# TEXT PREPROCESSING--> STEMMING, REMOVAL OF STOP WORDS, CONVERTING INTO LOWER CASE
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = [] 
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['content'][i])
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

corpus[1]

from tensorflow.keras.preprocessing.text import one_hot
onehot_repr= [one_hot(words,voc_size)for words in corpus]
print(onehot_repr[0])

# SETTING OUR SENTENCE LENGTH = 100
sent_length=100
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)
len(embedded_docs)

## CREATING OUR LSTM MODEL
embedding_vector_features= 40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length)) # making embedding layer
model.add(LSTM(100))  # one LSTM Layer with 100 neurons
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

# CONVERTING X AND Y INTO ARRAYS
x_final= np.array(embedded_docs)
y_final= np.array(y)
x_final.shape, y_final.shape


# SPLITTING THE DATA INTO TRAINING AND TEST SETS
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x_final,y_final,test_size=0.33,random_state=0)

print(x_train.shape, y_train.shape , x_test.shape, y_test.shape)

# FITTING NTO THE MODEL
model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=10,batch_size=64, verbose = True)

# OUR PREDICTION VARIABLE
y_pred= model.predict_proba(x_test)
y_pred = [int(i > .5) for i in y_pred]

# PRINTING CONFUSION MATRIX
# IMPORTING LIBRARIES TO SEE THE ACCURACY
from sklearn.metrics import confusion_matrix, accuracy_score
cm= confusion_matrix(y_test,y_pred)
print(cm)

# ACCURACY SCORE
ac= classification_report(y_test,y_pred)
print(ac)

#from keras.models import load_model

m#odel.save('./kaggle/working/my_model.h5')  # creates a HDF5 file 'my_model.h5'

(0, 2)
(42590, 2)
(14031, 2)
(0, 2)
(90904, 2)
(145517, 1)
(145517,)


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jupyter/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


KeyboardInterrupt: 